In [1]:
import numpy as np

from scipy.constants import e, c

from scipy.constants import physical_constants

import os

In [2]:
from cpymad.madx import Madx

import sixtracklib as stl
import pysixtrack

# Versioning

## SixTrackLib

In [3]:
f = os.path.dirname(stl.__file__)
f

'/home/oeftiger/gsi/git/sixtracklib_aoeftiger/python/sixtracklib'

In [4]:
!cd $f && git log | head -4

commit f42c420da20602c9c4c08172338ed3019bb20d25
Merge: deff9373 20f49532
Author: Adrian Oeftiger <a.oeftiger@gsi.de>
Date:   Wed Jan 8 14:52:08 2020 +0100


## PySixTrack

In [5]:
f = os.path.dirname(pysixtrack.__file__)
f

'/home/oeftiger/gsi/git/pysixtrack_aoeftiger/pysixtrack'

In [6]:
!cd $f && git log | head -4

commit c6e17f35b2f56e72bd65105907bd1bf5a8dcfc5f
Merge: a5079d5 a1c73d9
Author: Adrian Oeftiger <a.oeftiger@gmx.net>
Date:   Mon Jan 13 12:02:32 2020 +0100


# Parameters

In [7]:
nmass = physical_constants['atomic mass constant energy equivalent in MeV'][0] * 1e-3

In [8]:
A = 238
Q = 28

Ekin_per_nucleon = 0.2e9 # in eV

###

mass = A * nmass * 1e9 * e / c**2 # in kg
charge = Q * e # in Coul

Ekin = Ekin_per_nucleon * A
p0c = np.sqrt(Ekin**2 + 2*Ekin*mass/e * c**2) # in eV

Etot = np.sqrt(p0c**2 + (mass/e)**2 * c**4) * 1e-9 # in GeV
p0 = p0c / c * e # in SI units
gamma = np.sqrt(1 + (p0 / (mass * c))**2)
beta = np.sqrt(1 - gamma**-2)

## MAD-X setup

In [9]:
madx = Madx()
madx.options.echo = False
madx.options.warn = False


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.05.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2019.06.07               +
  + Execution date: 2020.01.15 11:57:49      +
  ++++++++++++++++++++++++++++++++++++++++++++


In [10]:
madx.input('''SET, format="22.14e";''')

True

In [11]:
# madx.input('OPTION, THIN_FOC = false;')

In [12]:
madx.input('''
qf: multipole, knl={1e-2};

testing: sequence, l = 1;
qf, at = 0.5;
endsequence;
''')

True

In [13]:
madx.command.beam(
    particle='ion', mass=A*nmass, charge=Q, energy=Etot)

True

In [14]:
madx.use(sequence='testing')

In [15]:
twiss_no_error = madx.twiss(betx=1, bety=1, dx=0, dy=0);

enter Twiss module

++++++ table: summ

                length                 orbit5                   alfa                gammatr 
  1.00000000000000e+00  -0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00 

                    q1                    dq1                betxmax                  dxmax 
  1.25000000000000e-01   0.00000000000000e+00   2.00000000000000e+00   8.80762727122517e-03 

                 dxrms                 xcomax                 xcorms                     q2 
  5.57043259177703e-03   0.00000000000000e+00   0.00000000000000e+00   1.25000000000000e-01 

                   dq2                betymax                  dymax                  dyrms 
  0.00000000000000e+00   2.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00 

                ycomax                 ycorms                 deltap                synch_1 
  0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00 

               synch_2   

In [16]:
q1_no_error = twiss_no_error.summary.q1

In [17]:
madx.select(flag='error', pattern='qf')

True

$\implies$ make the horizontal dipole a vertical one by turning by $90\deg$

In [18]:
madx.command.ealign(dpsi=np.pi/2)

Assigned alignment errors to 1 elements


True

In [19]:
# qf = madx.sequence.testing.elements[1] 
### --> only ".elements" does not work!! need "expanded_elements"
qf = madx.sequence.testing.expanded_elements[2]
qf

qf: multipole, at=0.5, knl={0.01};

In [20]:
qf.align_errors.dpsi

1.5707963267948966

In [21]:
twiss_with_error = madx.twiss(betx=1, bety=1, dx=0, dy=0);

enter Twiss module

++++++ table: summ

                length                 orbit5                   alfa                gammatr 
  1.00000000000000e+00  -0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00 

                    q1                    dq1                betxmax                  dxmax 
  1.25003979039367e-01   0.00000000000000e+00   1.99990000583337e+00   3.46944695195361e-18 

                 dxrms                 xcomax                 xcorms                     q2 
  2.19427091786044e-18   5.00016667333360e-03   3.16238307364026e-03   1.25003979039367e-01 

                   dq2                betymax                  dymax                  dyrms 
  0.00000000000000e+00   1.99990000583337e+00   8.80806767093869e-03   5.57071112501215e-03 

                ycomax                 ycorms                 deltap                synch_1 
  5.00041669208471e-03   3.16254119938251e-03   0.00000000000000e+00   0.00000000000000e+00 

               synch_2   

In [22]:
q1_with_error = twiss_with_error.summary.q1

$\implies$ **The tune has changed due to the weak focusing of the dipole:**

In [23]:
1 - q1_no_error / q1_with_error

3.1831301671947365e-05

## MAD-X tracking

In [24]:
np.sin(1e-2) - 1e-2

-1.6666583333574403e-07

Starting on-axis in the origin at (0,0,0,0,0,0), the particle gets a finite kick and drifts away from the axis:

In [25]:
madx.input('''
TRACK, onepass, onetable, file=output.;

START, x=0, px=0, y=0, py=0, t=0, pt=0;

RUN, turns=1;

ENDTRACK;
''')

enter TRACK module
one pass is on

++++++ table: tracksumm

    number       turn                      x                     px 
         1          0   0.00000000000000e+00   0.00000000000000e+00 
         1          1   5.00066682337395e-03   1.00003333466672e-02 

                     y                     py                      t                     pt 
  0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00 
 -5.00091686713360e-03  -1.00008333841694e-02  -8.80997659244605e-05   0.00000000000000e+00 

                     s                      e 
  0.00000000000000e+00   0.00000000000000e+00 
  1.00000000000000e+00   0.00000000000000e+00 
exit TRACK module



True

$\implies$ however, the kick in `px` and `py` is not identical to $|k_0 l|$ but has a $10^{-5}$ error!

In [26]:
headers = list(np.genfromtxt(
    "output.one", skip_header=51, max_rows=1, dtype=str)[1:])

initial_distribution_madx = np.genfromtxt(
    "output.one", skip_header=54, max_rows=1, dtype=np.float64)

final_distribution_madx = np.genfromtxt(
    "output.one", skip_header=55, max_rows=1, dtype=np.float64)

In [27]:
def get_betai(PT, mass=mass, p0=p0):
    restmass = mass * c**2
    restmass_sq = restmass**2
    E0 = np.sqrt((p0 * c)**2 + restmass_sq)
    
    E = E0 + PT * p0 * c
    gammai = E / restmass
    betai = np.sqrt(1 - 1. / (gammai * gammai))
    return betai

In [28]:
pt_madx = final_distribution_madx[headers.index('PT')]
betai = get_betai(pt_madx)

x_madx = final_distribution_madx[headers.index('X')]
px_madx = final_distribution_madx[headers.index('PX')]
y_madx = final_distribution_madx[headers.index('Y')]
py_madx = final_distribution_madx[headers.index('PY')]
zeta_madx = final_distribution_madx[headers.index('T')] * betai
delta_madx = np.sqrt(1 + 2 * pt_madx / beta + pt_madx**2) - 1

The $10^{-5}$ error (it actually depends on the initial strength, if the dipole gets stronger, the error becomes larger!):

In [29]:
1 - px_madx / qf.knl[0]

-3.333466672006402e-05

## PySixTrack setup

In [30]:
madx.esave(file='errors')
madx.command.readtable(file='errors', table='errors')
errors = madx.table.errors

Want to make named table: errors


In [31]:
# PySixTrack, lattice transfer and preparation!

pysixtrack_elements = pysixtrack.Line.from_madx_sequence(
    madx.sequence.testing, exact_drift=True,
)

pysixtrack_elements.apply_madx_errors(errors)

pysixtrack_elements.remove_zero_length_drifts(inplace=True);
pysixtrack_elements.merge_consecutive_drifts(inplace=True);

The PySixTrack multipole contains only the dipole component:

In [32]:
pysixtrack_elements.elements[2].knl

[0.01]

The `SRotation` wrapping the thin dipole applies the angle of $90\deg$:

In [33]:
angle = pysixtrack_elements.elements[1].angle
angle

90.0000000000002

The thin dipole contains a curvature part which has also been rotated:

In [34]:
pysixtrack_elements.elements[2].hxl

-3.4914813388431335e-17

In [35]:
pysixtrack_elements.elements[2].hyl

0.01

## PySixTrack tracking

In [36]:
pysixtrack_particles = pysixtrack.Particles.from_madx_track(madx)

In [37]:
# necessary to transfer to SixTrackLib with this version of PySixTrack
pysixtrack_particles.state = np.array([1, 0])
pysixtrack_particles.elemid = 0

pysixtrack_particles.remove_lost_particles()

In [38]:
## SixTrackLib stuff

# elements = stl.Elements.from_line(pysixtrack_elements)
# elements.BeamMonitor(num_stores=1);

# particles = stl.ParticlesSet().Particles(num_particles=1)

# particles.from_pysixtrack(pysixtrack_particles, particle_index=0)

# trackjob = stl.TrackJob(elements, particles) #, device="opencl:0.0")

# trackjob.track_until(1)
# trackjob.collect()

# x = trackjob.output.particles[0].x[0]
# px = trackjob.output.particles[0].px[0]
# y = trackjob.output.particles[0].y[0]
# py = trackjob.output.particles[0].py[0]
# zeta = trackjob.output.particles[0].zeta[0]
# delta = trackjob.output.particles[0].delta[0]

# x, px, y, py, zeta, delta

In [39]:
pysixtrack_particles

        mass0   = 221695594.70520002
        p0c     = 152876357.28239682
        energy0 = 269295594.7052
        beta0   = 0.5676897813711055
        gamma0  = 1.2147088220823519
        s       = [0.]
        x       = [0.]
        px      = [0.]
        y       = [0.]
        py      = [0.]
        zeta    = [0.]
        delta   = [0.]
        ptau    = [0.]
        mratio  = 1.0
        qratio  = 1.0
        chi     = 1.0

In [40]:
pysixtrack_elements.track(pysixtrack_particles)

In [41]:
pysixtrack_particles

        mass0   = 221695594.70520002
        p0c     = 152876357.28239682
        energy0 = 269295594.7052
        beta0   = 0.5676897813711055
        gamma0  = 1.2147088220823519
        s       = [1.]
        x       = [0.0050005]
        px      = [0.01]
        y       = [-0.0050005]
        py      = [-0.01]
        zeta    = [-5.00075013e-05]
        delta   = [0.]
        ptau    = [0.]
        mratio  = 1.0
        qratio  = 1.0
        chi     = 1.0

In [42]:
x = pysixtrack_particles.x[0]
px = pysixtrack_particles.px[0]
y = pysixtrack_particles.y[0]
py = pysixtrack_particles.py[0]
zeta = pysixtrack_particles.zeta[0]
delta = pysixtrack_particles.delta[0]

## evaluation PySixTrack vs. MAD-X:

In [43]:
print ('x error in %: {}'.format(100 * (x - x_madx) / x))

x error in %: -0.0033346337152075295


In [44]:
print ('px error in %: {}'.format(100 * (px - px_madx) / px))

px error in %: -0.0033334666716538946


In [45]:
print ('y error in %: {}'.format(100 * (y - y_madx) / y))

y error in %: -0.008335008796041616


In [46]:
print ('py error in %: {}'.format(100 * (py - py_madx) / py))

py error in %: -0.008333841693998478


In [47]:
print ('zeta error in %: {}'.format(100 * (zeta - zeta_madx) / zeta))

zeta error in %: -0.011669461951892845


In [48]:
delta == delta_madx

True

# Alternative by rotating k0l

In [49]:
# PySixTrack, lattice transfer and preparation!

pysixtrack_elements = pysixtrack.Line.from_madx_sequence(
    madx.sequence.testing, exact_drift=True,
)

# pysixtrack_elements.apply_madx_errors(errors)

pysixtrack_elements.remove_zero_length_drifts(inplace=True);
pysixtrack_elements.merge_consecutive_drifts(inplace=True);

Alternatively to rotating the coordinate system, we can rotate the knl and ksl (0-order) components to cross-check the approach:

In [50]:
multip = pysixtrack_elements.elements[1]
k0l = multip.knl[0]
multip.ksl[0] = k0l * np.sin(-angle * np.pi / 180.)
multip.knl[0] = k0l * np.cos(angle * np.pi / 180.)
multip

Multipole(knl=[-3.4914813388431335e-17], ksl=[-0.01], hxl=0.01, hyl=0.0, length=0.0)

In [51]:
pysixtrack_particles = pysixtrack.Particles.from_madx_track(madx)

In [52]:
# necessary to transfer to SixTrackLib with this version of PySixTrack
pysixtrack_particles.state = np.array([1, 0])
pysixtrack_particles.elemid = 0

pysixtrack_particles.remove_lost_particles()

In [53]:
pysixtrack_particles

        mass0   = 221695594.70520002
        p0c     = 152876357.28239682
        energy0 = 269295594.7052
        beta0   = 0.5676897813711055
        gamma0  = 1.2147088220823519
        s       = [0.]
        x       = [0.]
        px      = [0.]
        y       = [0.]
        py      = [0.]
        zeta    = [0.]
        delta   = [0.]
        ptau    = [0.]
        mratio  = 1.0
        qratio  = 1.0
        chi     = 1.0

In [54]:
pysixtrack_elements.track(pysixtrack_particles)

In [55]:
pysixtrack_particles

        mass0   = 221695594.70520002
        p0c     = 152876357.28239682
        energy0 = 269295594.7052
        beta0   = 0.5676897813711055
        gamma0  = 1.2147088220823519
        s       = [1.]
        x       = [0.0050005]
        px      = [0.01]
        y       = [-0.0050005]
        py      = [-0.01]
        zeta    = [-5.00075013e-05]
        delta   = [0.]
        ptau    = [0.]
        mratio  = 1.0
        qratio  = 1.0
        chi     = 1.0

In [56]:
x_alt = pysixtrack_particles.x[0]
px_alt = pysixtrack_particles.px[0]
y_alt = pysixtrack_particles.y[0]
py_alt = pysixtrack_particles.py[0]
zeta_alt = pysixtrack_particles.zeta[0]
delta_alt = pysixtrack_particles.delta[0]

## evaluation PySixTrack vs. MAD-X:

In [57]:
print ('x error in %: {}'.format(100 * (x_alt - x_madx) / x_alt))

x error in %: -0.0033346337152075295


In [58]:
print ('px error in %: {}'.format(100 * (px_alt - px_madx) / px_alt))

px error in %: -0.0033334666716538946


In [59]:
print ('y error in %: {}'.format(100 * (y_alt - y_madx) / y_alt))

y error in %: -0.008335008796041616


In [60]:
print ('py error in %: {}'.format(100 * (py_alt - py_madx) / py_alt))

py error in %: -0.008333841693998478


In [61]:
print ('zeta error in %: {}'.format(100 * (zeta_alt - zeta_madx) / zeta_alt))

zeta error in %: -0.011669461951892845


In [62]:
delta == delta_madx

True

## evaluation (i.) `SRotation` + `hxl`, `hyl` curvature rotation vs. (ii.) `knl`, `ksl` rotation

In [63]:
print ('x error in %: {}'.format(100 * (x_alt - x) / x_alt))

x error in %: 0.0


In [64]:
print ('px error in %: {}'.format(100 * (px_alt - px) / px_alt))

px error in %: 0.0


In [65]:
print ('y error in %: {}'.format(100 * (y_alt - y) / y_alt))

y error in %: -0.0


In [66]:
print ('py error in %: {}'.format(100 * (py_alt - py) / py_alt))

py error in %: -0.0


In [67]:
print ('zeta error in %: {}'.format(100 * (zeta_alt - zeta) / zeta_alt))

zeta error in %: -0.0


In [68]:
delta == delta_alt

True

$\implies$ the two approaches give identical results so they're equivalent. However, the more useful approach in the case of higher-order components of the `Multipole` is to use `SRotation` and rotate `hxl` and `hyl`